# CarND-Kidnapped-Vehicle-Project
The text in this file explains the functions of the **CarND-Kidnapped-Vehicle-Project** project (written in CPP).


Up on the reception of the message from the driving simulator (term2_sim.x86_64) h.onMessage (main.cpp) the following happens:
1. **init** if the particle filter is not initialized ```(!pf.initialized())``` initialize the filter particles with help of the current GPS position ```pf.init(sense_x, sense_y, sense_theta, sigma_pos)``` (actually GPS like position, x, y are in Cartesian cordinates not latitude and longitude) <br>
--> initialize values for x, y, and tetha (heading) based on a normal (Gaussian) distribution around x,y, and tetha<br>
--> set number of particle
2. if the particle filter is initialized do the rest of the items (3., 4., ...)
3. **prediction** uses above **Bicycle Model** to update the the position of the particles for each time step.
4. **updateWeights** updates the weights for each particle based on the observations (measurements to the land marks) and map. <br>
 For each particle: 
 * convert observations to map coordinate system (world cordinates) using **Conversion Formula** below
 * select predicted landmarks that are in range of the sensor 
 * Associate each observation to coresponding landmark ```dataAssociation```
 * using **Multi Variate Probability Density Function** (**Particle's Final Weight** below) ```multiv_prob``` calculate weight for all observations 
 * Normalize the weights
5. **resample** draw new particles (with replacement) with probability proportional to their weight

# Bicycle Model

if yaw rate is zero $\dot{\Theta}=0$ then <br>
$x_{f}=x_{0}+v*dt*cos(\Theta_{0})$ <br>
$y_{f}=y_{0}+v*dt*sin(\Theta_{0})$ <br>
$\Theta_{f}=\Theta_{0}$ <br>
 <br>
Otherwise if yaw rate is not zero $\Theta_{f}\neq=\Theta_{0}$ then <br>
$x_{f}=x_{0}+\frac{v}{\dot{\Theta}}*\left[sin(\Theta_{0}+\dot{\Theta}*dt)-sin(\Theta_{0})\right]$ <br>
$y_{f}=y_{0}+\frac{v}{\dot{\Theta}}*\left[cos(\Theta_{0})-cos(\Theta_{0}+\dot{\Theta}*dt)\right]$ <br>
$\Theta_{f}=\Theta_{0}+\dot{\Theta}*dt$ <br>
<br>
where <br>
$\Theta_{f}$ --  final heading angle <br>
$x_{f}$ -- final x position <br>
$y_{f}$ --  is final y position <br>
$\dot{\Theta}$ -- yaw rate <br>
$x_{0}$ -- initial x position <br>
$y_{0}$ -- initial y position <br>
$\Theta_{0}$ -- initial heading angle <br>
$dt$ -- delta time <br>


# Conversion Formula (Homogenous Transformation)
formula for converting car coordinate system $x_c,y_c$ to map coordinate system $x_m,y_m$ <br>
This formula includes translation and rotation <br>

\begin{equation*}
\begin{bmatrix}
x_m\\ 
y_m\\ 
1
\end{bmatrix}
=
\begin{bmatrix}
cos(\theta) & -sin(\theta) & x_p\\ 
sin(\theta) & cos(\theta) & y_p\\ 
0 & 0 & 1
\end{bmatrix}
\times 
\begin{bmatrix}
x_c\\ 
y_c\\ 
1
\end{bmatrix}
\end{equation*}




\begin{equation*}
V_{map}
=
C
\times 
V_{pEstimate}
\end{equation*}

$x_m=x_p + (cos\theta \times x_c) − (sin\theta \times y_c)$

$y_m=y_p + (sin\theta \times x_c) + (cos\theta \times y_c)$
<img src="localization-map-concept.png" height="420" width="420">

# Particle's Final Weight
Multivariate-Gaussian probability density 
\begin{equation*}
P(x,y) = \frac{1}{2\pi\sigma_x\sigma_y}e^{-({\frac{(x-\mu_x)^2}{2\sigma_x^2}+\frac{(y-\mu_y)^2}{2\sigma_y^2}} )}
\end{equation*}
where<br>
$P(x,y)$ = Particle Weight <br>
$x,y$ =  observations in map coordinates from the landmarks (transformed measurement's position) <br>
$\sigma_x\sigma_y$ = standard deviation for x,y <br>
$\mu_x, \mu_y$ are the coordinates of the nearest landmarks
